In [2]:
# Generate latlon table (very slow)
import time
import json
import pandas as pd

from IPython.display import display, clear_output

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Dataset path
dataset_path = "7mm_companies.csv"

# Read dataset
df = pd.read_csv(dataset_path)

# Set rate limiting
geolocator = Nominatim(user_agent='myapplication')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

# Parse all localities
localities = df['locality'].unique()


count = 0
pass_count = 0
fail_count = 0
locality_to_latlon = {}
failed_localities = []

t0 = time.time()

# Look up coordinates for all localities:
for locality in localities:
    count += 1
    try:
        location = geolocator.geocode(locality)
        latlon = (location.raw['lat'], location.raw['lon'])
        locality_to_latlon[locality] = latlon
        pass_count += 1
    except:
        failed_localities.append(locality)
        fail_count += 1
        
    clear_output(wait=True)
    print("{}/{} \tsuccess: {} \t fail: {}".format(count, localities.shape[0], pass_count, fail_count))
    
# Save to successful lookups:
with open('locality_to_latlon.json', 'w', encoding='utf-8') as f:
    json.dump(locality_to_latlon, f, ensure_ascii=False)

# Save failed lookups:
with open("failed_localities.txt", 'w') as f:
    for locality in failed_localities:
        f.write("{}\n".format(locality))
        
t1 = time.time()

# Print summary
print("Done")
print("Number of localities successfully found: {}".format(pass_count))
print("Number of localities not found: {}".format(fail_count))
print("Took {:0.2} seconds".format(t1-t0))

96245/96245 	success: 90274 	 fail: 5971
Done
Number of localities successfully found: 90274
Number of localities not found: 5971
Took 5e+04 seconds
